# Zillow Logerror Estimation Model

## Project Goals
1. create a model that can predict error in Zestimated values

2. from that model, identify key features driving logerror

3. clearly communicate finding to classmates

## Project Requirements
1. utilize clustering algorithms at some point in the pipeline

2. utilize statistical testing to identify key features

3. provide helpful visulatizations explaining exploration process

4. use scaling methods on data and document why they we're used

5. impute missing values and document

6. encode cateorical data

7. feature engineering and document rational behind it


### 0. Imports

In [3]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#python libraries used
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#project specific functions
import acquire
import prepare
import explore
import modeling
import cluster
from env import host,password,user



### I. Acquisition

Get data on homes sold in 2017 with recorded Latitude and Longitude

In [ ]:
#mysql query
query = '''
    SELECT prop.*, pred.logerror, pred.transactiondate
    FROM predictions_2017 AS pred
    LEFT JOIN properties_2017 AS prop  USING(parcelid)
    WHERE (bedroomcnt > 0 AND bathroomcnt > 0 AND calculatedfinishedsquarefeet > 500 AND latitude IS NOT NULL AND longitude IS NOT NULL) 
    AND (unitcnt = 1 OR unitcnt IS NULL)
    ;
    '''
db_name = 'zillow'
#get data from database
zillow = pd.read_sql(query, f"mysql+pymysql://{user}:{password}@{host}/{db_name}")

## II. Exploration